Dans cet atelier nous allons entrainer un modèle machine learning avec Scikit-Learn et puis le déployer en tant que web service local et aussi en tant que web service ACI "Azure Container Instance". 

Les étapes de cet Atelier sont les suivantes: 

1. entrainer localement un modèle Scikit-learn
2. Suivre les expérimentations Scikit-learn avec MLFlow sur Azure Machine Learning 
3. Enregistrer le modèle sur Azure Machine Learning 
4. Déployer et tester le modèle en tant que web service local 
5. Déployer et tester le modèle en tant que web service ACI "Azure Container Instance"

Tout au long de cet atelier nous allons utiliser Pima Indians Diabetes Database. Pour de plus amples informations sur les colonnes, visitez le lien ci-dessous: 

https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.

## 1. L'entrainement du modèle Scikit-Learn 

Assurez vous que vous avez installé anaconda avec une version de python 3.7.*

Assurez vous aussi que vous avez Docker installé : https://docs.docker.com/desktop/install/windows-install/ 

Dans le dossier de l'atelier, installez les dépendances requises: pip install -r requirements.txt 

* azureml-core==1.39
* pandas==1.3.5
* scikit-learn==0.24.2
* cloudpickle==2.0.0
* psutil==5.9.0
* mlflow==1.24.0

In [4]:
!pip show azureml-core

Name: azureml-core
Version: 1.54.0
Summary: Azure Machine Learning core packages, modules, and classes
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: 
License: https://aka.ms/azureml-sdk-license
Location: /home/ewins/anaconda3/lib/python3.9/site-packages
Requires: adal, argcomplete, azure-common, azure-core, azure-graphrbac, azure-mgmt-authorization, azure-mgmt-containerregistry, azure-mgmt-keyvault, azure-mgmt-network, azure-mgmt-resource, azure-mgmt-storage, backports.tempfile, contextlib2, docker, humanfriendly, jmespath, jsonpickle, knack, msal, msal-extensions, msrest, msrestazure, ndg-httpsclient, packaging, paramiko, pathspec, pkginfo, PyJWT, pyopenssl, python-dateutil, pytz, requests, SecretStorage, urllib3
Required-by: azureml-pipeline-core, azureml-sdk, azureml-telemetry, azureml-train-automl-client, azureml-train-core


In [1]:
import mlflow
from azureml.core import Workspace
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer

In [2]:
print(mlflow.__version__)


2.8.1


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#ws = Workspace.from_config() 
subscription_id = '2c2d1cc7-f0fc-4174-b403-4bdd35238850'
resource_group = 'Learn_MLOps'
workspace_name = 'MLOps_WS'
#workspace_location="France Central"
ws = Workspace.create(name = workspace_name,resource_group = resource_group,
                             subscription_id = subscription_id,exist_ok=True)

#### Mettre en place un URI de Tracking des experimentation MLflow sur Azure

Nous indiquons à mlflow que l'URI de tracking est celui de mlflow dans azure ML. Toutefois, le lien de tracking de Mlflow dans azure à la forme suivante:

azureml://<region>.api.azureml.ms/mlflow/v1.0/subscriptions/<subscription-id>/resourceGroups/<resource-group>/providers/Microsoft.MachineLearningServices/workspaces/<aml-workspace>?

In [4]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [5]:
mlflow.get_tracking_uri()

'azureml://francecentral.api.azureml.ms/mlflow/v2.0/subscriptions/2c2d1cc7-f0fc-4174-b403-4bdd35238850/resourceGroups/Learn_MLOps/providers/Microsoft.MachineLearningServices/workspaces/MLOps_WS?'

Nous allons créer une expérimentation Mlflow dont le nom est "diabetes-sklearn" 

In [6]:
experiment_name = 'weather-sklearn'
mlflow.set_experiment(experiment_name)

2023/11/19 22:37:28 INFO mlflow.tracking.fluent: Experiment with name 'weather-sklearn' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1700429849236, experiment_id='19647fd0-7c68-46ed-9a22-8862df49f082', last_update_time=None, lifecycle_stage='active', name='weather-sklearn', tags={}>

#### chargement du dataset et traitements necéssaires 

In [7]:
df=pd.read_csv('weather_dataset_processed.csv')

In [8]:
df.shape

(96449, 10)

In [9]:
df.columns

Index(['Timestamp', 'Location', 'Temperature_C', 'Humidity', 'Wind_speed_kmph',
       'Wind_bearing_degrees', 'Visibility_km', 'Pressure_millibars',
       'Current_weather_condition', 'Future_weather_condition'],
      dtype='object')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96449 entries, 0 to 96448
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Timestamp                  96449 non-null  object 
 1   Location                   96449 non-null  object 
 2   Temperature_C              96449 non-null  float64
 3   Humidity                   96449 non-null  float64
 4   Wind_speed_kmph            96449 non-null  float64
 5   Wind_bearing_degrees       96449 non-null  int64  
 6   Visibility_km              96449 non-null  float64
 7   Pressure_millibars         96449 non-null  float64
 8   Current_weather_condition  96449 non-null  int64  
 9   Future_weather_condition   96449 non-null  int64  
dtypes: float64(5), int64(3), object(2)
memory usage: 7.4+ MB


In [11]:
df.head()

,Timestamp,Location,Temperature_C,Humidity,Wind_speed_kmph,Wind_bearing_degrees,Visibility_km,Pressure_millibars,Current_weather_condition,Future_weather_condition
0,2006-04-01 04:00:00+02:00,"Port of Turku, Finland",8.755556,0.83,11.0446,259,15.8263,1016.51,1,1
1,2006-04-01 05:00:00+02:00,"Port of Turku, Finland",9.222222,0.85,13.9587,258,14.9569,1016.66,1,1
2,2006-04-01 06:00:00+02:00,"Port of Turku, Finland",7.733333,0.95,12.3648,259,9.9820,1016.72,1,1
3,2006-04-01 07:00:00+02:00,"Port of Turku, Finland",8.772222,0.89,14.1519,260,9.9820,1016.84,1,1
4,2006-04-01 08:00:00+02:00,"Port of Turku, Finland",10.822222,0.82,11.3183,259,9.9820,1017.37,1,1


In [12]:
X = df[['Temperature_C', 'Humidity', 'Wind_speed_kmph', 'Wind_bearing_degrees', 'Visibility_km', 'Pressure_millibars', 'Current_weather_condition']].values
y = df['Future_weather_condition'].values
y

array([1, 1, 1, ..., 0, 0, 0])

In [13]:
# Splitting the Training dataset into Train and Test set for ML training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
X_test

array([[-0.14529951, -1.56468002, -0.64169187, ..., -0.31725673,
         0.19516031,  0.41651557],
       [-0.19886072, -1.61591723, -0.38295596, ...,  1.37284272,
         0.21155394,  0.41651557],
       [-0.82762267,  0.4848081 , -0.1568534 , ...,  0.91190651,
         0.10909373,  0.41651557],
       ...,
       [-1.6496707 ,  0.58728251, -0.43889886, ..., -1.03170787,
         0.24972037, -2.40087065],
       [ 1.48424188, -1.92334045,  0.74056394, ..., -0.08678863,
         0.14230792,  0.41651557],
       [ 2.02101829, -1.35973121,  0.96899744, ...,  0.20513764,
         0.07869721,  0.41651557]])

In [17]:
print("X_train:",X_train.shape)
print("X_test:",X_test.shape)
print("Y_train:",y_train.shape)
print("Y_test:",y_test.shape)

X_train: (77159, 7)
X_test: (19290, 7)
Y_train: (77159,)
Y_test: (19290,)


In [18]:
rf_clf = RandomForestClassifier() 

Au lieu d'appeler a chaque fois une fonction de logging (log_metric, log_artifact, ...) nous allons utiliser la fonction autolog() afin de logger toutes les informations relatives au modèle (metrics, parameters, ...) 

les frameworks suivants supporte la fonction autolog():

    Scikit-learn

    TensorFlow and Keras

    Gluon

    XGBoost

    LightGBM

    Statsmodels

    Spark

    Fastai

    Pytorch



In [19]:
mlflow.sklearn.autolog(max_tuning_runs=None)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f5a9ec63d30>
Traceback (most recent call last):
  File "/home/ewins/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ewins/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ewins/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ewins/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


#### Hyperparameter tuning 

Nous alons utiliser l'algorithme GridSearchCV pour trouver les meilleurs paramétres

In [20]:
param_grid = {'n_estimators': [10,20,30], 'max_depth':[2,7,10]}

In [21]:
gridcv=GridSearchCV(rf_clf,param_grid=param_grid,scoring = ['roc_auc', 'precision', 'recall', 'f1', 'accuracy'],refit = 'roc_auc',cv=10)

In [23]:
gridcv.fit(X_train, y_train)

2023/11/19 22:49:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '29324c38-d59d-4197-9605-28d00b121221', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 7, 10], 'n_estimators': [10, 20, 30]},
             refit='roc_auc',
             scoring=['roc_auc', 'precision', 'recall', 'f1', 'accuracy'])

## 2.  L'enregistrement du modèle 
L'enregistrement du modèle dans le registre de modèles d'Azure Machine Learning a pour but de permettre aux utilisateurs de suivre les modifications apportées au modèle via la gestion des versions du modèle.

nous allons trouver l'expérimentation à partir du workspace en définant le workspace et le nom de l'expérimentation 

In [24]:
from azureml.core import Experiment, Workspace
experiment_name = 'weather-sklearn'
experiment = Experiment(ws, experiment_name)

In [25]:
# afficher tous les runs 
for r in experiment.get_runs():
    print(r)

Run(Experiment: weather-sklearn,
Id: 29324c38-d59d-4197-9605-28d00b121221,
Type: None,
Status: Completed)
Run(Experiment: weather-sklearn,
Id: 5e7cf57e-de44-4afe-87f7-5059b07e7829,
Type: None,
Status: Failed)


Trouver le Run actuel 

In [26]:
run_id = '29324c38-d59d-4197-9605-28d00b121221'
run = [r for r in experiment.get_runs() if r.id == run_id][0]

Enregistrer le modèle 

    model_name: un nom arbitraire pour enregister le modèle 
    model_path: chemin vers  model.pkl

In [27]:
model = run.register_model(model_name = 'weather_model', model_path = 'best_estimator/model.pkl')

## 3. la création d'un script de Scoring

Le script de scoring généralement appelé score.py est utilisé lors de l'inférence comme point d'entrée du modèle.

score.py (voir le dossier de l'atelier) consiste en deux fonctions obligatoires:

1. init(): charge le modèle en tant que variable globale
2. run(): reçoit les nouvelles données à prédire à travers le paramètre data
      * effectue un pré-traitement des nouvelles données (optionnel)
      * effectue une prédiction sur les nouvelles données
      * effectue un post-traitement sur les prédictions (optionnel)
      * renvoie les résultats de la prédiction

## 4. Déploiement en local

Maintenant nous allons debuger le web service locallement avant de le déployer en production avec ACI " Azure container Instance"

Récupérez le modèle enregistré en définissant le workspace, le nom du modèle et la version du modèle.

In [28]:
from azureml.core.model import Model
model = Model(ws, 'weather_model', version=1)

In [30]:
from azureml.core import Environment
env = Environment.from_conda_specification(name='weather-env', file_path="./conda.yaml")
env.register(ws)

{
    "assetId": "azureml://locations/francecentral/workspaces/aa1e5275-5e6b-4445-acff-433f66377ce9/environments/weather-env/versions/2",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20231030.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "weather-env",
    "

### Définons la configuration d'inférence 

In [31]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="./score.py",
)

### Définons la configuration de déploiement

In [34]:
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=6799)

### Déployons le service localement

Avant d'exécuter la cellule ci-dessous, assurez vous que Docker est démarré 

In [35]:
service = Model.deploy(
    workspace = ws,
    name = 'weather-prediction-service',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry aa1e52755e6b4445acff433f66377ce9.azurecr.io
Logging into Docker registry aa1e52755e6b4445acff433f66377ce9.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM aa1e52755e6b4445acff433f66377ce9.azurecr.io/azureml/azureml_b51cc7c611c5465dbbb1f942c06bdac1
 ---> 3b5a950429c7
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 08455fffbc69
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjJjMmQxY2M3LWYwZmMtNDE3NC1iNDAzLTRiZGQzNTIzODg1MCIsInJlc291cmNlR3JvdXBOYW1lIjoibGVhcm5fbWxvcHMiLCJhY2NvdW50TmFtZSI6Im1sb3BzX3dzIiwid29ya3NwYWNlSWQiOiJhYTFlNTI3NS01ZTZiLTQ0NDUtYWNmZi00MzNmNjYzNzdjZTkifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 7c476c14e30e
 ---> 37ee1c6d4585
Step 4/5 : RUN mv '/var/azureml-app/tmplpuueaas.py' /var/azureml-app/main.py
 ---> Running in f7f883100400
 ---> 

In [36]:
service.wait_for_deployment(show_output=True)

Checking container health...
Local webservice is running at http://localhost:6799


Le fichier model.pkl sera téléchargé à partir d'Azure Machine Learning dans un dossier local temporaire et une image Docker avec les dépendances est créée et enregistrée dans Azure Container Registry (ACR). L'image sera téléchargée d'ACR sur la machine locale et un conteneur Docker exécutant le service Web est construit à partir de l'image localement.



Pour avoir l'URI de scoring lancez la commande suivante 

In [37]:
print (service.scoring_uri)

http://localhost:6799/score


## 5. Testons le service localement 

Pour tester le service localement, ouvrez le notebook "Inference_test.ipynb" et assigner l'URI de scroring à la variable scoring_uri

Nous avons envoyé une demande d'inference au scoring_uri avec les données au format JSON. Voici à quoi ressemble input_data :

{"input": "[{\"Pregnancies\":6,\"Glucose\":148,\"BloodPressure\":72,\"SkinThickness\":35,\"Insulin\":0,\"BMI\":33.6,\"DiabetesPedigreeFunction\":0.627,\"Age\":50}]"}


Voici un exemple de la réponse pour l'inférence sur un seul enregistrement. La valeur de retour contient la probabilité qu'une personne reçoive un diagnostic de diabète.

prediction: "{\"proba\": [0.4829951216899814]}"


Le format de réponse peut être personnalisé dans la fonction run du fichier score.py.

## 6. Déplyons le service sur ACI

In [38]:
from azureml.core import Workspace
ws = Workspace.from_config()
from azureml.core.model import Model
model = Model(ws, 'weather_model', version=1)
from azureml.core import Environment
env = Environment.get(workspace = ws, name = 'weather-env', version = 1)

In [39]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="./score.py")

In [40]:
from azureml.core.webservice import AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=0.1, memory_gb=0.5, auth_enabled=False)

In [41]:
service = Model.deploy(
    workspace = ws,
    name = 'weather-prediction-service',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)
    
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-11-19 23:31:04+01:00 Creating Container Registry if not exists.
2023-11-19 23:31:04+01:00 Registering the environment.
2023-11-19 23:31:06+01:00 Use the existing image.
2023-11-19 23:31:06+01:00 Generating deployment configuration.
2023-11-19 23:31:07+01:00 Submitting deployment to compute.
2023-11-19 23:31:11+01:00 Checking the status of deployment weather-prediction-service..
2023-11-19 23:32:43+01:00 Checking the status of inference endpoint weather-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
#service.delete()

In [42]:
print (service.scoring_uri)

http://a1ed98f6-478c-4825-91b0-bd37c3eb6cea.francecentral.azurecontainer.io/score


### Testons le service 

Refaites la même chose en ouvrant le notebook "Inference_test.ipynb" et assigner le nouveau URI de scroring à la variable scoring_uri 

## Déploiement sur AKS "Azure Kubernetes Service" 

ACI est recommandé pour les tests sur une petite charge de travail de production. Pour une charge de travail importante azure fourni AKS "Azure Kubernetes Service"

Malheureusement le Déploiement sur AKS n'est pas pris en compte  avec notre sousription actuelle. 

Néanmoins, je vous ai fourni le code de déploiement pour une éventuelle utilisation future

In [ ]:
from azureml.core.webservice import AksWebservice
deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=False)

In [ ]:
# ça suppose la création d'un kubernetes cluster 
from azureml.core.compute import AksCompute
aks_target = AksCompute(ws,"myaks")

In [ ]:
service = Model.deploy(
    workspace = ws,
    name = 'diabetes-prediction-service-aks',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    deployment_target=aks_target,
    overwrite=True)
    
service.wait_for_deployment(show_output=True)